In [3]:
import pandas as pd
from janitor import clean_names
import janitor

In [2]:
annotation_srcs =  {
    "aug_liftoff_cds_agree_use_liftoff": "liftoff",
    "aug_liftoff_cds_disagree_use_aug": "augustus",
    "aug_liftoff_pasa_agree_subgene_tama_merges_use_pasa": "pasa",
    "aug_liftoff_pasa_cds_agree_use_pasa": "pasa",
    "aug_pasa_cds_agree_use_pasa": "pasa",
    "aug_pasa_cds_disagree_but_cds_too_long_for_isoseq_use_aug": "augustus",
    "liftoff_augustus_agree_subgene_isoseq_merges_use_liftoff": "liftoff",
    "only_aug": "augustus",
    "only_liftoff": "liftoff",
    "pasa_cds_different_use_pasa": "pasa",
    "pasa_liftoff_cds_agree_use_pasa": "pasa",
    "TOO_HARD": "manual"}


In [12]:
a = (pd.read_csv("https://docs.google.com/spreadsheets/d/1rlotaQ-u83HLd5ZGjR-UnCxC2Hzg8gzEWCGy37s_J7c/gviz/tq?tqx=out:csv&sheet=All",
                 keep_default_na=False)
     .clean_names()
     .assign(accession = lambda x: [n.split("_")[0] if isinstance(n, str) else None for n in x.chromosome])
     .filter(["accession", "chromosome", "locus_start", "locus_end", "locus_name", "strand",
             "locus_coords", "kevin_decision", "pseudogene", "manual_annotation_status",
             "readiness", "annotation_source"])
    )

In [13]:
a

,accession,chromosome,locus_start,locus_end,locus_name,strand,locus_coords,kevin_decision,manual_annotation_status,readiness,annotation_source
0,at6137,at6137_1_chr1,1586677,1587586,gene00471,+,at6137_1_chr1:1586678-1587586,aug_liftoff_cds_disagree_use_aug,,,
1,at6137,at6137_1_chr1,3127863,3128456,gene00933,+,at6137_1_chr1:3127864-3128456,aug_liftoff_cds_disagree_use_aug,,,
2,at6137,at6137_1_chr1,3639291,3642258,gene01075,-,at6137_1_chr1:3639292-3642258,aug_liftoff_cds_disagree_use_aug,,,
3,at6137,at6137_1_chr1,4133814,4136472,gene01225,+,at6137_1_chr1:4133815-4136472,aug_liftoff_cds_agree_use_liftoff,,,
4,at6137,at6137_1_chr1,4137431,4140771,gene01227,+,at6137_1_chr1:4137432-4140771,aug_liftoff_cds_agree_use_liftoff,,,
...,...,...,...,...,...,...,...,...,...,...,...
4603,at9879,at9879_1_chr5,29690003,29692924,gene34198,-,at9879_1_chr5:29690004-29692924,aug_pasa_cds_agree_use_pasa,,,
4604,at9879,at9879_1_chr5,30995338,30996032,gene34596,+,at9879_1_chr5:30995339-30996032,aug_liftoff_cds_agree_use_liftoff,,,
4605,at9879,at9879_1_chr5,31009502,31015599,gene34601,+,at9879_1_chr5:31009503-31015599,TOO_HARD,doing,,
4606,at9879,at9879_1_chr5,31132195,31135324,gene34634,-,at9879_1_chr5:31132196-31135324,aug_liftoff_pasa_cds_agree_use_pasa,,,


In [5]:

core_decisions = a %>%
    transmute(accession=sub("_1_chr.", "", chromosome), chromosome, locus_start, locus_end,
              locus_name, strand, locus_coords, kevin_decision,
              pseudogene,
        manual_annotation_status = sub("already ", "", manual_annotation_status),
        readiness = case_when(
            kevin_decision=="TOO_HARD" & manual_annotation_status == "done"    ~ "R",
            kevin_decision=="TOO_HARD" & manual_annotation_status == "doing"   ~ "NR",
            T ~ readiness,
            ),
        annotation_source = case_when(
            !is.na(annotation_source) ~ annotation_source,
            readiness == "R" & kevin_decision %in% names(annotation_srcs)               ~ annotation_srcs[kevin_decision]
        )
    ) %>% 
    filter(!is.na(locus_name))

,number,luisa_id,Chromosome,locus_start,locus_end,locus_name,Unnamed: 6,Strand,locus_coords,Chrom_gene,...,NLRanno_type,Augustus IDs,Orthogroup,Kevin decision,Manual annotation status,Notes,Pseudogene?,Readiness,annotation_source,luisa_fix_me_please
0,1.0,Loci_1-at6137_1_chr1-1586677-1587586,at6137_1_chr1,1586677.0,1587586.0,gene00471,0.0,+,at6137_1_chr1:1586678-1587586,at6137_1_chr1-gene00471,...,NaN,g452,OG0000712,aug_liftoff_cds_disagree_use_aug,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Loci_2-at6137_1_chr1-3127863-3128456,at6137_1_chr1,3127863.0,3128456.0,gene00933,0.0,+,at6137_1_chr1:3127864-3128456,at6137_1_chr1-gene00933,...,TIR,g896,OG0004959,aug_liftoff_cds_disagree_use_aug,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,Loci_3-at6137_1_chr1-3639291-3642258,at6137_1_chr1,3639291.0,3642258.0,gene01075,0.0,-,at6137_1_chr1:3639292-3642258,at6137_1_chr1-gene01075,...,CC-NBARC-LRR,g1035,OG0019720,aug_liftoff_cds_disagree_use_aug,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,Loci_4-at6137_1_chr1-4133814-4136472,at6137_1_chr1,4133814.0,4136472.0,gene01225,0.0,+,at6137_1_chr1:4133815-4136472,at6137_1_chr1-gene01225,...,NBARC-LRR,g1176,OG0002416,aug_liftoff_cds_agree_use_liftoff,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,Loci_5-at6137_1_chr1-4137431-4140771,at6137_1_chr1,4137431.0,4140771.0,gene01227,0.0,+,at6137_1_chr1:4137432-4140771,at6137_1_chr1-gene01227,...,CC-NBARC-LRR,g1177,OG0000160,aug_liftoff_cds_agree_use_liftoff,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4603,NaN,NaN,at9879_1_chr5,29690003.0,29692924.0,gene34198,0.0,-,at9879_1_chr5:29690004-29692924,NaN,...,NaN,NaN,NaN,aug_pasa_cds_agree_use_pasa,NaN,NaN,NaN,NaN,NaN,NaN
4604,NaN,NaN,at9879_1_chr5,30995338.0,30996032.0,gene34596,0.0,+,at9879_1_chr5:30995339-30996032,NaN,...,NaN,NaN,NaN,aug_liftoff_cds_agree_use_liftoff,NaN,NaN,NaN,NaN,NaN,NaN
4605,NaN,NaN,at9879_1_chr5,31009502.0,31015599.0,gene34601,0.0,+,at9879_1_chr5:31009503-31015599,NaN,...,NaN,NaN,NaN,TOO_HARD,doing,take transdecoder and liftoff,NaN,NaN,NaN,NaN
4606,NaN,NaN,at9879_1_chr5,31132195.0,31135324.0,gene34634,0.0,-,at9879_1_chr5:31132196-31135324,NaN,...,NaN,NaN,NaN,aug_liftoff_pasa_cds_agree_use_pasa,NaN,NaN,NaN,NaN,NaN,NaN
